# ICS mitre mapping

This notebook attempts to visualize the mapping between several ICS threat actors and conducted attacks to the detections as to discover the attack.

This notebook uses data from the MITRE ATT&CK ICS framework using the MITRE STIX data in the [attack-stix-data](https://github.com/mitre-attack/attack-stix-data/tree/master) repository.

## Requirements

There are some libraries that we need and with which the code was tested

* **taxii2-client**: To talk to the taxii server
* **stix2**: To handle the stix2 encapsulated data
* **networkx**: To create and work with graphs
* **bokeh**: To visualize graphs
* **pygraphviz**: Additional library to structure the graph

**NOTE:** Pygraphviz is actually a base package that needs to be installed via the terminal, for Ubuntu and other debian based systems run ```sudo apt-get install graphviz graphviz-dev```.

For other systems consult [pygraphviz install guide](https://github.com/pygraphviz/pygraphviz/blob/main/INSTALL.txt) 

In [10]:
%pip install taxii2-client==2.3.0
%pip install stix2==3.0.1
%pip install networkx==2.6.3
%pip install bokeh==2.3.3
%pip install pandas==1.3.5
%pip install pygraphviz==1.12

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Initial setup and retrieving data
The debugging level is lowered to only show critical messages, and we use the following url to work with: 
```https://cti-taxii.mitre.org/taxii/```


In [11]:
from taxii2client.v20 import Server

import logging
logging.getLogger('taxii2client').setLevel(logging.CRITICAL)
server = Server("https://cti-taxii.mitre.org/taxii/")

# Retrieve API root
api_root = server.api_roots[0]

# Select ICS ATT&CK data collection
MITRE_COLLECTION_ID = api_root.collections[3].id

Now get the actual collection, based on the selected collection ID

In [12]:
from stix2 import TAXIICollectionSource, Filter
from taxii2client.v20 import Collection

ATTACK_STIX_COLLECTIONS = "https://cti-taxii.mitre.org/stix/collections/"
MITRE_COLLECTION = Collection(ATTACK_STIX_COLLECTIONS + MITRE_COLLECTION_ID + "/")
MC_DATA = TAXIICollectionSource(MITRE_COLLECTION)

Also get the mitigation (course of action), detections, attacker groups, software and technique data. We retreive it as a mapping of unique IDs and human readable names

In [13]:
MC_MITIGATIONS = MC_DATA.query(Filter("type", "=", "course-of-action"))
mc_mitigation_mapping = {}
for x in MC_MITIGATIONS:
  mc_mitigation_mapping[x['id']] = x['name']

In [14]:
MC_TECHNIQUES = MC_DATA.query(Filter("type", "=", "attack-pattern"))
mc_technique_mapping = {}
for x in MC_TECHNIQUES:
  mc_technique_mapping[x['id']] = x['name']

In [15]:
MC_DETECTIONS = MC_DATA.query(Filter("type", "=", "x-mitre-data-component"))
mc_detection_mapping = {}
for x in MC_DETECTIONS:
  mc_detection_mapping[x['id']] = x['name']

In [16]:
MC_INTRUSION_SET = MC_DATA.query(Filter("type", "=", "intrusion-set"))
mc_intrusion_set_mapping = {}
for x in MC_INTRUSION_SET:
  mc_intrusion_set_mapping[x['id']] = x['name']

In [17]:
MC_MALWARE = MC_DATA.query(Filter("type", "=", "malware"))
mc_malware_mapping = {}
for x in MC_MALWARE:
  mc_malware_mapping[x['id']] = x['name']

Let's now get the relationships between threat actors, software, techniques, and detections.
The data is then put into a pandas dataframe with the human readable names as well for better context

In [18]:
import pandas as pd

MC_RELATIONS = MC_DATA.query(Filter("type","=","relationship"))

mc_ot_data = {'Source':[],'Destination':[],'source_human':[],'target_human':[]}
for idx,x in enumerate(MC_RELATIONS):
  if 'relationship_type' in x:
    if x['relationship_type'] == 'uses' and 'intrusion-set' in x['source_ref'] and 'malware' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_intrusion_set_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_malware_mapping[x['target_ref']])
    elif x['relationship_type'] == 'uses' and 'malware' in x['source_ref'] and 'attack-pattern' in x['target_ref']:
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_malware_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
    elif x['relationship_type'] == 'detects':
      mc_ot_data['Source'].append(x['source_ref'])
      mc_ot_data['Destination'].append(x['target_ref'])
      mc_ot_data['source_human'].append(mc_detection_mapping[x['source_ref']])
      mc_ot_data['target_human'].append(mc_technique_mapping[x['target_ref']])
      

df = pd.DataFrame(data=mc_ot_data)
df.head()

,Source,Destination,source_human,target_human
0,intrusion-set--381fcf73-60f6-4ab2-9991-6af3cbc...,malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5,Sandworm Team,Industroyer2
1,malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5,attack-pattern--8e7089d3-fba2-44f8-94a8-9a79c5...,Industroyer2,Brute Force I/O
2,malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5,attack-pattern--097924ce-a9a9-4039-8591-e0deed...,Industroyer2,Modify Parameter
3,malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5,attack-pattern--40b300ba-f553-48bf-862e-9471b2...,Industroyer2,Unauthorized Command Message
4,malware--6a0d0ea9-b2c4-43fe-a552-ac41a3009dc5,attack-pattern--2d0d40ad-22fa-4cc8-b264-072557...,Industroyer2,Monitor Process State


## Visualizing all data

In [19]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html
from bokeh.io import output_notebook, show, save
# required to display visuals inline
output_notebook()

Loading BokehJS ...

In [32]:


from bokeh.core.property_mixins import LineJoin
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap
from bokeh.models import EdgesAndLinkedNodes, NodesAndLinkedEdges
import networkx as nx


DG = nx.from_pandas_edgelist(df, source='Source', target='Destination',create_using=nx.Graph())

# set the degrees (connections per node)
degrees = dict(nx.degree(DG))
nx.set_node_attributes(DG, name='degree', values=degrees)

#set the human readable attribute
all_mappings = {}
all_mappings.update(mc_technique_mapping)
all_mappings.update(mc_mitigation_mapping)
all_mappings.update(mc_detection_mapping)
all_mappings.update(mc_intrusion_set_mapping)
all_mappings.update(mc_malware_mapping)

nx.set_node_attributes(DG,name='human_name',values=all_mappings)

#ensure very small degrees are still visible
number_to_adjust_by = 5
adjusted_node_size = dict([(node, degree+number_to_adjust_by) for node, degree in nx.degree(DG)])
nx.set_node_attributes(DG, name='adjusted_node_size', values=adjusted_node_size)

# determine type, add attribute
node_types = {}

for node in DG.nodes:
    if node.startswith('intrusion-set'):
      node_types[node] = 0
    elif node.startswith('malware'):
      node_types[node] = 1 
    elif node.startswith('attack-pattern'):
      node_types[node] = 2
    else:
      node_types[node] = 3

#set the node type (attack-pattern,course-of-action,data-source)
nx.set_node_attributes(DG, name='node_type', values=node_types)

#Choose attributes from G network to size and color by — setting manual size (e.g. 10) or color (e.g. 'skyblue') also allowed
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'node_type'

#color_palette = red,green,blue
color_palette = ['#000000','#FF0000','#00FF00','#0000FF']

#Choose colors for node and edge highlighting
node_highlight_color = 'white'
edge_highlight_color = 'black'

#Choose a title!
title = 'MITRE Graph Visuals'

#Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Name", "@human_name"),("Degree", "@degree")]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

#Create a network graph object with spring layout
# https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html
network_graph = from_networkx(DG, nx.spring_layout, scale=10, center=(0, 0))
#network_graph = from_networkx(DG, nx.kamada_kawai_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = 0
maximum_value_color = 2
#Set node size and color
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set node highlight colors
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, fill_color=node_highlight_color, line_width=2)


#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Set edge highlight colors
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width=2)

#Highlight nodes and edges
network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()


#Add network graph to the plot
plot.renderers.append(network_graph)
plot.plot_width = 1100
plot.plot_height = 550
show(plot)
#save(plot, filename=f"{title}.html")

## Further refinement of visualization (not functional yet)

To do further visualization, we'll convert the NetworkX graph to a PyGraphViz object and then some arranging of the different node/vertex types into columns for a better connection overview.

In [29]:
import pygraphviz as pgv

A = nx.nx_agraph.to_agraph(DG)

show()

ValueError: "Invalid object to show. The object to passed to show must be one of:

* a LayoutDOM (e.g. a Plot or Widget or Layout)
* a Bokeh Application
* a callable suitable to an application FunctionHandler
